In [2]:
# REQUIREMENTS:
# pip install reportlab pillow qrcode

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.lib import colors
from reportlab.lib.utils import ImageReader
from datetime import datetime
from io import BytesIO
import qrcode
import os, math

# ------------------ Helpers ------------------
def _fit(text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    s = max_font
    while s > min_font and pdfmetrics.stringWidth(t, font_name, s) > box_w:
        s -= 0.5
    return max(min_font, s)

def _draw_scaled_title(c, text, page_w, top_y, side_margin, font_bold="Helvetica-Bold",
                       max_font=18, min_font=10):
    box_w = page_w - 2*side_margin
    fs = _fit(text, font_bold, max_font, min_font, box_w)
    c.setFont(font_bold, fs)
    c.setFillColor(colors.black)
    c.drawCentredString(page_w/2, top_y, text)
    return fs

def _draw_qr_centered(c, data, page_w, top_y, size_mm=24, gap_mm=2):
    if not data:
        return top_y
    qr = qrcode.QRCode(border=2, box_size=8)
    qr.add_data(str(data))
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    bio = BytesIO()
    img.save(bio, format="PNG")
    bio.seek(0)
    ir = ImageReader(bio)

    size = size_mm * mm
    x = (page_w - size) / 2.0
    y = top_y - size - (gap_mm*mm)
    c.drawImage(ir, x, y, width=size, height=size, preserveAspectRatio=True, mask='auto')
    return y  # returns y-position just below the QR

def _draw_field_row(c, top_y, height, left, usable_w, pad, cols, prefill,
                    label_fonts=("Helvetica","Helvetica-Bold"),
                    label_sizes=(10,7.2), value_font_max=11.5):
    c.setLineWidth(1.05)
    c.setStrokeColor(colors.black)
    c.rect(left, top_y-height, usable_w, height, stroke=1, fill=0)

    x = left; acc = 0.0
    for i,(field,ratio) in enumerate(cols):
        acc += ratio
        if i < len(cols)-1:
            x_next = left + usable_w*acc
            c.line(x_next, top_y-height, x_next, top_y)

        col_w = usable_w*ratio
        label = field.get("label","")
        key   = field.get("key","")
        fs = _fit(label, label_fonts[0], label_sizes[0], label_sizes[1], col_w-2*pad)
        c.setFont(label_fonts[0], fs); c.setFillColor(colors.black)
        c.drawString(x+pad, top_y - pad - fs*0.2, label)

        val = str(prefill.get(key,""))
        if val:
            vfs = _fit(val, label_fonts[1], value_font_max, 8, col_w-2*pad)
            c.setFont(label_fonts[1], vfs)
            y_val = top_y - max(6.8*mm, min(height-3.0*mm, height*0.55))
            c.drawString(x+pad, y_val, val)
        x = left + usable_w*acc

def _fmt_money(v):
    try:
        return f"{float(v):,.2f}"
    except:
        return str(v)

def _draw_table_header(c, x, y_top, col_ws, row_h, headers, font_bold="Helvetica-Bold"):
    total_w = sum(col_ws)
    c.setLineWidth(0.9)
    c.setStrokeColor(colors.black)
    c.rect(x, y_top - row_h, total_w, row_h, stroke=1, fill=0)
    cx = x
    for cw in col_ws[:-1]:
        cx += cw
        c.line(cx, y_top - row_h, cx, y_top)
    c.setFont(font_bold, 8.6)
    cur_x = x
    for i,h in enumerate(headers):
        c.drawString(cur_x + 1.8*mm, y_top - row_h + 2.6*mm, str(h))
        cur_x += col_ws[i]

def _draw_table_rows(c, x, y_top, col_ws, row_h, rows):
    total_w = sum(col_ws)
    n = len(rows)
    c.setLineWidth(0.8); c.setStrokeColor(colors.black)
    c.rect(x, y_top - n*row_h, total_w, n*row_h, stroke=1, fill=0)
    cx = x
    for cw in col_ws[:-1]:
        cx += cw
        c.line(cx, y_top - n*row_h, cx, y_top)
    for r in range(1, n):
        y = y_top - r*row_h
        c.line(x, y, x + total_w, y)

    for r, row in enumerate(rows):
        baseline = y_top - r*row_h - (row_h*0.68)
        cur_x = x
        for i, cw in enumerate(col_ws):
            text = "" if i >= len(row) or row[i] is None else str(row[i])
            avail = cw - 2.6*mm
            # --- 50% larger font for Item No. column (i == 0) ---
            max_base, min_base = 8.2, 6.2
            scale = 1.5 if i == 0 else 1.0
            fs = _fit(text, "Helvetica", max_base*scale, min_base*scale, avail)
            c.setFont("Helvetica", fs)
            c.drawString(cur_x + 1.8*mm, baseline, text)
            cur_x += cw

def _paginate_table(c, page_w, page_h, margins_mm, start_cursor, col_mm, headers, row_h_mm, rows,
                    title_for_cont="SALES INVOICE FORM (cont.)", label_fonts=("Helvetica","Helvetica-Bold"),
                    qr_cfg=None):
    qr_cfg = qr_cfg or {}
    m = margins_mm*mm
    left = m
    col_ws = [wmm*mm for wmm in col_mm]
    row_h = row_h_mm*mm
    cursor = start_cursor
    i = 0
    while i < len(rows):
        min_needed = row_h * 2.0
        if (cursor - m) < min_needed:
            c.showPage()
            _draw_scaled_title(c, title_for_cont, page_w, page_h - m - 5*mm, m, label_fonts[1], 16, 9)
            qr_data = qr_cfg.get("data")
            qr_size_mm = qr_cfg.get("size_mm", 24)
            qr_gap_mm = qr_cfg.get("gap_mm", 2)
            cursor = _draw_qr_centered(c, qr_data, page_w, page_h - m - 8*mm, size_mm=qr_size_mm, gap_mm=qr_gap_mm)
            cursor -= 6*mm
        _draw_table_header(c, left, cursor, col_ws, row_h, headers, label_fonts[1])
        cursor -= row_h
        max_rows_here = max(1, int(math.floor((cursor - m) / row_h)))
        end = min(len(rows), i + max_rows_here)
        _draw_table_rows(c, left, cursor, col_ws, row_h, rows[i:end])
        cursor -= (end - i) * row_h
        i = end
    return cursor

# ------------------ Main renderer ------------------
def render_sales_invoice_form(cfg=None):
    cfg = cfg or {}
    title = cfg.get("title","SALES INVOICE FORM")
    filename = cfg.get("filename", os.path.join(os.getcwd(), "sales_invoice_form_a4.pdf"))
    page_size = cfg.get("page_size", A4)
    margins_mm = cfg.get("margins_mm", 6)
    pad_mm = cfg.get("pad_mm", 3)
    label_font = cfg.get("label_font", ("Helvetica","Helvetica-Bold"))
    label_font_sizes = cfg.get("label_font_sizes",(10,7.2))
    value_font_max = cfg.get("value_font_max", 11.5)
    title_font_sizes = cfg.get("title_font_sizes",(18,10))
    base_row_height_mm = cfg.get("table_row_h_mm", 8.2)
    extra_top_gap_mm = cfg.get("tables_extra_top_gap_mm", 6)
    qr_size_mm = cfg.get("qr_size_mm", 24)
    qr_gap_mm = cfg.get("qr_gap_mm", 2)
    qr_data_key = cfg.get("qr_data_key", "current_form_qr_code")
    force_empty_rows = int(cfg.get("force_empty_rows", 0))
    trailing_blank_rows = int(cfg.get("trailing_blank_rows", 2))

    w,h = page_size
    m = margins_mm*mm
    pad = pad_mm*mm
    left, right = m, w - m
    usable_w = right - left

    # ---- Header fields (includes QR chain fields) ----
    FIELDS = [
        ([({"key":"id","label":"Document ID"},0.34),
          ({"key":"tin","label":"TIN"},0.20),
          ({"key":"date","label":"Date (YYYY-MM-DD)"},0.18),
          ({"key":"created","label":"Created (ISO)"},0.28)],"single"),
        ([({"key":"previous_form_qr_code","label":"Previous Form QR Code"},0.50),
          ({"key":"current_form_qr_code","label":"Current Form QR Code"},0.50)],"single"),
        ([({"key":"location","label":"Location"},1.0)],"single"),
    ]
    HEIGHTS = {"single": 10*mm}

    p = dict(cfg.get("prefill",{}))
    prefill = {
        "id": p.get("id",""),
        "tin": p.get("tin",""),
        "date": p.get("date",""),
        "created": p.get("created",""),
        "location": p.get("location",""),
        "previous_form_qr_code": p.get("previous_form_qr_code",""),
        "current_form_qr_code": p.get("current_form_qr_code",""),
        "cashier_employee_number": p.get("cashier_employee_number",""),
        "recorder_employee_number": p.get("recorder_employee_number",""),
    }

    # Build rows with Item No. column (auto-numbered)
    items = list(p.get("items") or [])
    rows = []
    if force_empty_rows > 0:
        total_rows = force_empty_rows
        for i in range(1, total_rows+1):
            rows.append([str(i), "", "", "", "", "", ""])
    else:
        for i, it in enumerate(items, start=1):
            rows.append([
                str(i),                                     # Item No.
                it.get("receipt_number",""),
                it.get("customer_name",""),
                str(it.get("type","")).title(),
                _fmt_money(it.get("vatable_sales","")),
                _fmt_money(it.get("vat_amount","")),
                _fmt_money(it.get("total_amount","")),
            ])
        for j in range(1, trailing_blank_rows+1):
            rows.append([str(len(items)+j), "", "", "", "", "", ""])

    headers = ["Item No.","Receipt No.","Customer Name","Type","Vatable Sales","VAT","Total Amount"]

    # Column widths in mm, sum = 180 mm
    # [12, 44, 38, 18, 28, 20, 20] = 180
    col_mm  = [12, 44, 38, 18, 28, 20, 20]
    row_h_mm = base_row_height_mm

    c = canvas.Canvas(filename, pagesize=page_size)
    c.setTitle(title)

    # Title
    title_y = h - m - 5*mm
    _draw_scaled_title(c, title, w, title_y, m, label_font[1], title_font_sizes[0], title_font_sizes[1])

    # Centered QR below title
    qr_data = p.get(qr_data_key, "")
    cursor = _draw_qr_centered(c, qr_data, w, title_y - 2*mm, size_mm=qr_size_mm, gap_mm=qr_gap_mm)
    cursor -= 6*mm

    # Header fields
    for cols, hspec in FIELDS:
        h_row = HEIGHTS[hspec]
        if cursor - h_row < m + 24:
            c.showPage()
            _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
            cursor = _draw_qr_centered(c, qr_data, w, h - m - 8*mm, size_mm=qr_size_mm, gap_mm=qr_gap_mm)
            cursor -= 6*mm
        _draw_field_row(c, cursor, h_row, left, usable_w, pad, cols, prefill,
                        label_fonts=label_font, label_sizes=label_font_sizes, value_font_max=value_font_max)
        cursor -= h_row

    # Table label + spacing
    cursor -= (extra_top_gap_mm*mm)
    c.setFont(label_font[1], 9.2)
    if force_empty_rows > 0:
        label_text = f"SALES INVOICE LINE ITEMS (blank form: {force_empty_rows} rows)"
    else:
        label_text = f"SALES INVOICE LINE ITEMS ({len(items)} items) + {trailing_blank_rows} allowance"
    c.drawString(left, cursor - 1.6*mm, label_text)
    cursor -= 5.8*mm

    if (cursor - m) < (row_h_mm*mm*2):
        c.showPage()
        _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
        cursor = _draw_qr_centered(c, qr_data, w, h - m - 8*mm, size_mm=qr_size_mm, gap_mm=qr_gap_mm)
        cursor -= 6*mm

    # Render table with pagination
    cursor = _paginate_table(
        c, w, h, margins_mm, cursor,
        col_mm, headers, row_h_mm, rows,
        title_for_cont=title + " (cont.)",
        label_fonts=label_font,
        qr_cfg={"data": qr_data, "size_mm": qr_size_mm, "gap_mm": qr_gap_mm}
    )

    # Footer: employee numbers
    cursor -= 10*mm
    footer_cols = [
        ({"key":"cashier_employee_number","label":"Cashier Employee Number"}, 0.50),
        ({"key":"recorder_employee_number","label":"Recorder Employee Number"}, 0.50),
    ]
    h_row = 14*mm
    if cursor - h_row < m + 10:
        c.showPage()
        _draw_scaled_title(c, title + " (cont.)", w, h - m - 5*mm, m, label_font[1], 16, 9)
        cursor = _draw_qr_centered(c, qr_data, w, h - m - 8*mm, size_mm=qr_size_mm, gap_mm=qr_gap_mm)
        cursor -= 6*mm

    _draw_field_row(c, cursor, h_row, left, usable_w, pad, footer_cols, prefill,
                    label_fonts=label_font, label_sizes=label_font_sizes, value_font_max=value_font_max)

    c.save()
    return os.path.abspath(filename)

# -------- Empty-form usage (20 blank rows, empty cashier/recorder) --------
empty_form = {
  "id": "",
  "current_form_qr_code": "",
  "previous_form_qr_code": "",
  "tin": "",
  "location": "",
  "created": "",
  "date": "",
  "items": [],
  "cashier_employee_number": "",
  "recorder_employee_number": ""
}

cfg = {
    "filename": "sales_invoice_form_empty_20rows_a4.pdf",
    "prefill": empty_form,
    "table_row_h_mm": 8.2,
    "qr_size_mm": 24,
    "qr_gap_mm": 2,
    "qr_data_key": "current_form_qr_code",
    "force_empty_rows": 20,          # EXACTLY 20 blank rows
    "trailing_blank_rows": 2         # used only when items are present
}

render_sales_invoice_form(cfg)


'h:\\github4\\jefstore-gasstations-backend\\reportlab\\sales-invoices\\empty\\sales_invoice_form_empty_20rows_a4.pdf'